In [1]:
# 202202015/마다현

# 1 파일을 읽고 데이터의 앞 부분을 출력하는 코드

import pandas as pd
import numpy as np
import tensorflow as tf

# df = pd.read_csv(r'C:\Users\CS3-23\Downloads\admission.csv')
df = pd.read_csv('/Users/dahyoun/Documents/coding/Python/MachineLearning/KBU_2024_ML/dataSource/admission.csv')
############### 내 경로로 임시 수정 ###############################
# df = pd.read_csv('./admission.csv')
###############################################################
df.head()

,GRE_Score,TOEFL_Score,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit
0,337.0,118,4,4.5,4.5,9.65,1,0.92
1,324.0,107,4,4.0,4.5,8.87,1,0.76
2,316.0,104,3,3.0,3.5,8.00,1,0.72
3,322.0,110,3,3.5,2.5,8.67,1,0.80
4,314.0,103,2,2.0,3.0,8.21,0,0.65


In [2]:
# 2 대학평가지수(University_Rating)에 따른 학부 성적의 평균을 구하는 코드

def custom_mean(group):
    return np.mean(group)

num2_ans = df.groupby('University_Rating')['CGPA'].agg(custom_mean)

print(num2_ans.to_string(header=False, float_format="{:.6f}".format))

1   7.745769
2   8.183738
3   8.552256
4   9.021622
5   9.291167


In [3]:
#3 연구 경험 유무에 따른 비율을 구하는 코드

research_counts = df['Research'].value_counts()
research_ratio = research_counts / research_counts.sum()

research_yes = research_ratio[1]
research_no = research_ratio[0]

print("연구경험 유: ", research_yes)
print("연구경험 무: ", research_no)

연구경험 유:  0.5475
연구경험 무:  0.4525


In [4]:
#4 결측치 비율을 확인하는 코드

num4_ans = df.isnull().mean()
num4_ans

GRE_Score            0.02
TOEFL_Score          0.00
University_Rating    0.00
SOP                  0.00
LOR                  0.00
CGPA                 0.00
Research             0.00
Chance_of_Admit      0.00
dtype: float64

In [5]:
# df.isna().sum(): DataFrame에서 각 열(칼럼)에 대해 결측값(NaN)개수를 계산하는 메소드

df.isna().sum()

GRE_Score            8
TOEFL_Score          0
University_Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance_of_Admit      0
dtype: int64

In [6]:
df.apply(lambda column: column.fillna(df.groupby('University_Rating')[column.name].transform('mean')) if column.isnull().any() else column)

,GRE_Score,TOEFL_Score,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit
0,337.0,118,4,4.5,4.5,9.65,1,0.92
1,324.0,107,4,4.0,4.5,8.87,1,0.76
2,316.0,104,3,3.0,3.5,8.00,1,0.72
3,322.0,110,3,3.5,2.5,8.67,1,0.80
4,314.0,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...
395,324.0,110,3,3.5,3.5,9.04,1,0.82
396,325.0,107,3,3.0,3.5,9.11,1,0.84
397,330.0,116,4,5.0,4.5,9.45,1,0.91
398,312.0,103,3,3.5,4.0,8.78,0,0.67


In [7]:
# 5 결축치가 있는 열에 대하여 대학평가지수에 따른 평균값으로 대체하고 다시 결측치 개수를 확인하는 코드

# num5_ans = df.apply(lambda column: column.fillna(df.groupby('University_Rating')[column.name].transform('mean')) if column.isnull().any() else column)

# num5_ans = num5_ans.isnull().sum()
# print(num5_ans)

######### 아에 원본 데이터 셋에서 NA를 제거해야함 -> 아니면 훈련시에 loss가 nan남 ##################
df = df.apply(lambda column: column.fillna(df.groupby('University_Rating')[column.name].transform('mean')) if column.isnull().any() else column)

print(df.isnull().sum())
######################################################################

GRE_Score            0
TOEFL_Score          0
University_Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance_of_Admit      0
dtype: int64


In [8]:
# 6 대학 평가 지수를 원 핫 인코딩하여 원본 데이터에 열 방향으로 합하고 앞 일부를 출력하는 코드 - prefix는 “UR”로 지정

one_hot_encoded = pd.get_dummies(df['University_Rating'], prefix='UR')

df = pd.concat([df, one_hot_encoded], axis=1)
df.head()

,GRE_Score,TOEFL_Score,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit,UR_1,UR_2,UR_3,UR_4,UR_5
0,337.0,118,4,4.5,4.5,9.65,1,0.92,False,False,False,True,False
1,324.0,107,4,4.0,4.5,8.87,1,0.76,False,False,False,True,False
2,316.0,104,3,3.0,3.5,8.00,1,0.72,False,False,True,False,False
3,322.0,110,3,3.5,2.5,8.67,1,0.80,False,False,True,False,False
4,314.0,103,2,2.0,3.0,8.21,0,0.65,False,True,False,False,False


In [9]:
# 7 대학원 입학 평가 시험 점수는 최솟값-최댓값 정규화하여 ‘GRE’열을 생성하고 TOEFL 점수는 z-스코어로 정규화하여 ‘TOEFL’열을 생성하여 앞 일부를 출력하는 코드

# MinMaxScaler를 사용하여 GRE 점수 정규화
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df['GRE'] = scaler.fit_transform(df[['GRE_Score']])

# z-스코어 정규화를 직접 계산하여 TOEFL 점수 정규화
toefl_mean = df['TOEFL_Score'].mean()
toefl_std = df['TOEFL_Score'].std()
df['TOEFL'] = (df['TOEFL_Score'] - toefl_mean) / toefl_std

# 앞 일부 데이터 출력
print(df[['GRE_Score', 'TOEFL_Score', 'University_Rating', 'SOP', 'LOR', 'CGPA', 'Research',
          'Chance_of_Admit', 'UR_1', 'UR_2', 'UR_3', 'UR_4', 'UR_5', 'GRE', 'TOEFL']].head())

   GRE_Score  TOEFL_Score  University_Rating  SOP  LOR  CGPA  Research  \
0      337.0          118                  4  4.5  4.5  9.65         1   
1      324.0          107                  4  4.0  4.5  8.87         1   
2      316.0          104                  3  3.0  3.5  8.00         1   
3      322.0          110                  3  3.5  2.5  8.67         1   
4      314.0          103                  2  2.0  3.0  8.21         0   

   Chance_of_Admit   UR_1   UR_2   UR_3   UR_4   UR_5   GRE     TOEFL  
0             0.92  False  False  False   True  False  0.94  1.744786  
1             0.76  False  False  False   True  False  0.68 -0.067551  
2             0.72  False  False   True  False  False  0.52 -0.561824  
3             0.80  False  False   True  False  False  0.64  0.426723  
4             0.65  False   True  False  False  False  0.48 -0.726582  


In [10]:
# 8 「Ouput 데이터 만들기」 대학원 합격 확률을 아래 표처럼 구간을 나누어 범주형 데이터로 생성하고 앞 일부를 출력하는 코드

# 대학원 합격 확률을 범주형 데이터로 변환
df['admit_Bad'] = (df['Chance_of_Admit'] <= 7.5)
df['admit_NotBad'] = ((df['Chance_of_Admit'] > 7.5) & (df['Chance_of_Admit'] <= 8.0))
df['admit_Okay'] = ((df['Chance_of_Admit'] > 8.0) & (df['Chance_of_Admit'] <= 8.5))
df['admit_Good'] = ((df['Chance_of_Admit'] > 8.5) & (df['Chance_of_Admit'] <= 9.0))
df['admit_Great'] = ((df['Chance_of_Admit'] > 9.0) & (df['Chance_of_Admit'] <= 9.5))
df['admit_Excellent'] = (df['Chance_of_Admit'] > 9.5)

# 결과 출력
print(df[['admit_Bad', 'admit_NotBad', 'admit_Okay', 'admit_Good', 'admit_Great',
          'admit_Excellent']].head())


   admit_Bad  admit_NotBad  admit_Okay  admit_Good  admit_Great  \
0       True         False       False       False        False   
1       True         False       False       False        False   
2       True         False       False       False        False   
3       True         False       False       False        False   
4       True         False       False       False        False   

   admit_Excellent  
0            False  
1            False  
2            False  
3            False  
4            False  


In [11]:
# 9 「데이터 분리」 테스트 데이터는 전체 데이터의 25%로 사용하여 훈련 데이터와 테스트 데이터를 분리하는 코드
# - 코드 분리 후, 넘파이 배열로 지정 후, “np.float32”로 데이터 타입 지정하는 것이 안정적

from sklearn.model_selection import train_test_split

# 특성 데이터와 타깃 데이터 분리
# X = df.drop(columns=['Chance_of_Admit'])
# y = df['Chance_of_Admit']
################## X = x_col의 열만 사용 #####################
# 기존코드는 불필요한 열이 많았음, 필요한 열만 사용해야 함
x_col=['University_Rating','SOP','LOR','CGPA','Research','UR_1','UR_2','UR_3','UR_4','UR_5','GRE','TOEFL']
X = df[x_col]
################## y = nx6 이어야 함#######################
# output보면 결과가 6개의 레이블(원핫인코딩된 chance_of_admit)이기 때문에, y가 그냥, 1차원 벡터(nx1)이면 안된다.
y = df[['admit_Bad', 'admit_NotBad', 'admit_Okay', 'admit_Good', 'admit_Great', 'admit_Excellent']]
y = y.values # 넘파이 배열로 변환 (필수)
#########################################################

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 데이터 타입 변환
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

In [12]:
################# 피처 수 확인 = 12개 ##########
X
##########################################

,University_Rating,SOP,LOR,CGPA,Research,UR_1,UR_2,UR_3,UR_4,UR_5,GRE,TOEFL
0,4,4.5,4.5,9.65,1,False,False,False,True,False,0.94,1.744786
1,4,4.0,4.5,8.87,1,False,False,False,True,False,0.68,-0.067551
2,3,3.0,3.5,8.00,1,False,False,True,False,False,0.52,-0.561824
3,3,3.5,2.5,8.67,1,False,False,True,False,False,0.64,0.426723
4,2,2.0,3.0,8.21,0,False,True,False,False,False,0.48,-0.726582
...,...,...,...,...,...,...,...,...,...,...,...,...
395,3,3.5,3.5,9.04,1,False,False,True,False,False,0.68,0.426723
396,3,3.0,3.5,9.11,1,False,False,True,False,False,0.70,-0.067551
397,4,5.0,4.5,9.45,1,False,False,False,True,False,0.80,1.415270
398,3,3.5,4.0,8.78,0,False,False,True,False,False,0.44,-0.726582


In [13]:
#################### 피처 수 확인 ################
y
##########################################

array([[ True, False, False, False, False, False],
       [ True, False, False, False, False, False],
       [ True, False, False, False, False, False],
       ...,
       [ True, False, False, False, False, False],
       [ True, False, False, False, False, False],
       [ True, False, False, False, False, False]])

In [14]:
# 10. 「모델 생성」 그림 “머신러닝 지도 학습 구조”를 기반으로 모델 생성하여 summary함수 호출하는 코드

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout

# 입력 특성의 수를 바로 모델의 입력층에 전달하여 모델 생성
input_layer = Input(shape=(X_train.shape[1],))

################## pdf 나온대로 Dense 층 만들기 ############################
dense_layer_1 = Dense(10, activation='relu')(input_layer)
dense_layer_2 = Dense(8, activation='swish')(dense_layer_1)
output = Dense(6, activation='softmax')(dense_layer_2)
##################################################################

# # 모델 생성
# dense_layer_1 = Dense(15, activation='relu')(input_layer)
# dense_layer_2 = Dense(10, activation='relu')(dense_layer_1)

# # 교수님 ppt
# output = Dense(1, activation='sigmoid')(dense_layer_2)
model = Model(inputs=input_layer, outputs=output, name='model')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# gpt 코드
# output = Dense(1, activation='sigmoid')(dense_layer_2)
# model = Model(inputs=input_layer, outputs=output, name='model')
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

# 모델 요약 출력
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12)]              0         
                                                                 
 dense (Dense)               (None, 10)                130       
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
 dense_2 (Dense)             (None, 6)                 54        
                                                                 
Total params: 272
Trainable params: 272
Non-trainable params: 0
_________________________________________________________________


In [15]:
# 11 「학습」 배치 사이즈는 5, 에폭은 10으로 하여 학습 시키는 코드

# from tensorflow.keras.optimizers import legacy
# optimizer = legacy.Adam()

history = model.fit(X_train, y_train, batch_size=5, epochs=10, verbose=1, validation_split=0.2)

Epoch 1/10


2024-04-19 13:02:07.592873: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


48/48 [==============================] - 0s 3ms/step - loss: 0.7960 - acc: 0.9000 - val_loss: 0.3489 - val_acc: 1.0000
Epoch 2/10
48/48 [==============================] - 0s 821us/step - loss: 0.1457 - acc: 1.0000 - val_loss: 0.0496 - val_acc: 1.0000
Epoch 3/10
48/48 [==============================] - 0s 731us/step - loss: 0.0296 - acc: 1.0000 - val_loss: 0.0185 - val_acc: 1.0000
Epoch 4/10
48/48 [==============================] - 0s 746us/step - loss: 0.0134 - acc: 1.0000 - val_loss: 0.0101 - val_acc: 1.0000
Epoch 5/10
48/48 [==============================] - 0s 1ms/step - loss: 0.0079 - acc: 1.0000 - val_loss: 0.0064 - val_acc: 1.0000
Epoch 6/10
48/48 [==============================] - 0s 738us/step - loss: 0.0052 - acc: 1.0000 - val_loss: 0.0045 - val_acc: 1.0000
Epoch 7/10
48/48 [==============================] - 0s 706us/step - loss: 0.0038 - acc: 1.0000 - val_loss: 0.0033 - val_acc: 1.0000
Epoch 8/10
48/48 [==============================] - 0s 756us/step - loss: 0.0029 - acc: 1.0

In [16]:
# 12 「평가」 Loss율과 정확도를 보여주는 코드

# 테스트 데이터에 대한 Loss와 정확도 계산
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

4/4 [==============================] - 0s 1ms/step - loss: 0.0015 - acc: 1.0000
Test Loss: 0.001486897119320929
Test Accuracy: 1.0
